In [1]:
import pandas as pd
import numpy as np
import Levenshtein as l

In [2]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200

In [3]:
class Prepack:
    """Data preparation library v0.1""" 
    
    def __init__(self):
        pass
    
    @staticmethod
    def isnan(v):
        import math as m
        if isinstance(v, float) and m.isnan(v):
            return True
        else:
            return False
    
    @staticmethod    
    def list_concat(lst1, lst2):

        lst3 = []
        size1 = len(lst1)
        size2 = len(lst2)
        #здесь будет размер самого длинного списка
        size_max = max(size1,size2)

        #вытаскиваем и делаем строковыми значения из обоих списков
        for i in range(size_max):
            ss = ''
            if i < size1 and not Prepack.isnan(lst1[i]):
                s = str(lst1[i])
                if len(s) > 0:
                    ss += s
            if i < size2 and not Prepack.isnan(lst2[i]):
                s = str(lst2[i])
                if len(s) > 0:
                    if len(ss) > 0:
                        ss += ' '
                    ss += s
            #кладем все в список в виде объединенных значений
            lst3 += [ss]
        return lst3
    
    @staticmethod 
    def p2c(lst):
        import numpy as np
        length = len(lst)
        max_len = max(len(repr(el)) for el in lst)
        # делаем кортеж со значениями из списка
        cols = tuple(lst)
        # делаем массив из кортежа названий и диапазона от 0 до length, .T транспонирует матрицу
        a = np.array([np.arange(length),cols]).T

        #смотрим форму массива, по оси 0 - это y, и по оси 1 - это x
        y, x = a.shape

        #берем половину массив, округляя в большую сторону
        col1 = a[:int(np.ceil(y/2))].copy()
        #оставщаяся часть
        col2 = a[int(np.ceil(y/2)):].copy()

        #определяем максимальную длину колонок, чтобы увеличить одну из колонок, если она получилась короче, например, когда 
        #кол-во столбцов было изначально 3, первая колонка будет длиной 2, а вторая 1
        length = max(col1.shape[0], col2.shape[0])

        #меняем размер колонок по максимальной длине
        col1.resize([length,2])
        col2.resize([length,2])

        #Приходится изворачиваться, у Питона нет ++/-- для инкремента/декремента. Те мелочи, за которые я не люблю Питон 
        i = 0
        while i < length:
            c1 = str(col1[i][0]).ljust(3) #первая колонка
            c1_len = len(c1) 
            c2 = repr(col1[i][1]).ljust(max_len+3) 
            c2_len = len(c2) 
            c3 = str(col2[i][0]).ljust(3) #вторая колонка
            c3_len = len(c3)
            c4 = repr(col2[i][1])
            print(c1 + c2 + c3 + c4)
            i += 1
    
    #тут пара функций для удобного сохранения из загрузки 
    @staticmethod 
    def load(filepath):
        import pickle as pkl
        with open(filepath, "rb") as f:
            return pkl.load(f)
    
    @staticmethod 
    def save(data, filepath):
        import pickle as pkl
        with open(filepath, "wb") as f:
            return pkl.dump(data, f, 2) #2 is protocol version
    
    @staticmethod 
    def read_excels(filepath):
        import pandas as pd
        return pd.read_excel(filepath, sheet_name=None, header=None, na_filter=False)

    @staticmethod 
    def read_excel(filepath):
        import pandas as pd
        return pd.read_excel(filepath, header=None, na_filter=False)

    @staticmethod 
    def read_zip(filepath):
        import zipfile as zip
        z = zip.ZipFile(filepath, mode='r')
        names = tuple(z.namelist())
        lst = []
        for f in names:
            lst += [z.open(f)]
        return (names, tuple(lst))
    
    @staticmethod 
    def df_filter_and(df, fltr):
        import numpy as np
        lst = []
        for col in fltr:
            if fltr[col] == 'isnum':
                lst.append(df.loc[:,col].astype(str).str.replace('.','').str.isnumeric())
            elif fltr[col] == 'isnotnum':
                lst.append(~df.loc[:,col].astype(str).str.replace('.','').str.isnumeric())
            else:
                lst.append(df.loc[:,col] == fltr[col])
                
        if len(lst) == 1:
            return lst[0]
        else:
            res = lst[0]
            for i in range(1,len(lst)):
                res = np.logical_and(res, lst[i])
            return res
    
    @staticmethod 
    def df_filter_or(df, fltr):
        import numpy as np
        lst = []
        for col in fltr:
            if fltr[col] == 'isnum':
                lst.append(df.loc[:,col].astype(str).str.replace('.','').str.isnumeric())
            elif fltr[col] == 'isnotnum':
                lst.append(~df.loc[:,col].astype(str).str.replace('.','').str.isnumeric())
            else:
                lst.append(df.loc[:,col] == fltr[col])
        
        if len(lst) == 1:
            return lst[0]
        else:
            res = lst[0]
            for i in range(1,len(lst)):
                res = np.logical_or(res, lst[i])
            return res

    #делает слияние двух датафреймов через наиболее близкое расстояние Левенштейна
    @staticmethod
    def levenstein_merge(dfa, dfb, left_on, right_on, limit = 0x7FFFFFFF): #0x7FFFFFFF is a max integer
        import Levenshtein as l
        a = dfa[left_on]
        b = dfb[right_on]

        mindst = 0x7FFFFFFF
        klist = [None] * len(a)
        lena = len(dfa.columns) 
        lenb = len(dfb.columns) 
        bcols = list(range(lena, lena+lenb)) #bcols is a list with dfb columns indexes after concatenation

        res = dfa
        to_concat = pd.DataFrame([], columns=dfb.columns)
        res = pd.concat([res, to_concat], axis=1) #this will add columns to res from dfb

        for i, ival in enumerate(a):
            mindst = 0x7FFFFFFF #reset mindst on each cycle
            for k, kval in enumerate(b):
                dst = l.distance(ival, kval)
                if dst <= limit and dst < mindst: # if current distance less than saved
                    mindst = dst
                    klist[i] = k
                    if mindst == 0: #stop if strings are equals
                        break

            if klist[i] != None: #if index found than
                res.iloc[i,bcols] = list(dfb.iloc[klist[i],:]) #set dfb columns values in res row to klist[i] row in dfb

        return res
    
pp = Prepack       

In [20]:
yy = pp.load('02-19.pkl')
cofog = pd.read_excel('cofog_rus.xlsx')

In [78]:
y = yy['2002']
for i in y.index:
    print(i, y.loc[i, 'Наименование'])

0 Государственное управление и местное самоуправление
1 Судебная власть
2 Международная деятельность
3 Национальная оборона
4 Правоохранительная деятельность и обеспечение безопасности государства
5 Фундаментальные исследования и содействие научно-техническому прогрессу
6 Промышленность, энергетика и строительство
7 Сельское хозяйство и рыболовство
8 Охрана окружающей природной среды и природных ресурсов, гидрометеорология, картография и геодезия
9 Транспорт, связь и информатика
10 Развитие рыночной инфраструктуры
11 Предупреждение и ликвидация последствий чрезвычайных ситуаций и стихийных бедствий
12 Образование
13 Культура, искусство и кинематография
14 Средства массовой информации
15 Здравоохранение и физическая культура
16 Социальная политика
17 Обслуживание государственного и муниципального долга
18 Пополнение государственных запасов и резервов
19 Финансовая помощь бюджетам других уровней
20 Исследование и использование космического пространства
21 Военная реформа
22 Дорожное хозя

In [23]:
cofog_ = cofog.drop(0).reset_index(drop=True)
cofog_

,code,name
0,701,Государственные службы общего назначения
1,7011,"Исполнительные и законодательные органы, бюдже..."
2,7012,Иностранная экономическая помощь
3,7013,Общие службы
4,7014,Фундаментальные исследования
5,7015,"НИОКР, связанные с государственными службами о..."
6,7016,"Государственные службы общего назначения, не о..."
7,7017,"Операции, связанные с государственным долгом"
8,7018,Трансферты общего характера между органами гос...
9,702,Оборона


In [97]:
pp.levenstein_merge( y, cofog_, 'Наименование', 'name2', 20)

,РЗ,Наименование,Сумма,code,name,name2
0,01,Государственное управление и местное самоуправ...,58140558.3,7011,"Исполнительные и законодательные органы, бюдже...",Государственное управление и местное самоуправ...
1,02,Судебная власть,19900896.6,7033,Суды,Судебная власть
2,03,Международная деятельность,41574577.2,7033,Суды,Судебная власть
3,04,Национальная оборона,2819655.7,702,Оборона,Национальная оборона
4,05,Правоохранительная деятельность и обеспечение ...,58375369.3,7031,Полицейские службы,Правоохранительная деятельность и обеспечение ...
5,06,Фундаментальные исследования и содействие науч...,29610544.9,7025,"Вопросы обороны, не отнесенные к другим катего...",Фундаментальные исследования и содействие науч...
6,07,"Промышленность, энергетика и строительство",103396876.6,7061,Жилищное строительство,"Промышленность, энергетика и строительство"
7,08,Сельское хозяйство и рыболовство,28876289.4,7042,"Сельское хозяйство, лесное хозяйство, рыболовс...",Сельское хозяйство и рыболовство
8,09,Охрана окружающей природной среды и природных ...,9956333.3,705,Охрана окружающей среды,Охрана окружающей природной среды и природных ...
9,10,"Транспорт, связь и информатика",7495681.6,704,Экономические вопросы,"Транспорт, связь и информатика"


In [98]:
cofog_['name2'] = cofog_['name']

cofog_.loc[18,'name2'] = 'Судебная власть'
cofog_.loc[17,'name2'] = 'Предупреждение и ликвидация последствий чрезвычайных ситуаций и стихийных бедствий'
cofog_.loc[9,'name2'] = ' Национальная оборона'
cofog_.loc[14,'name2'] = 'Фундаментальные исследования и содействие научно-техническому прогрессу'
cofog_.loc[16,'name2'] = 'Правоохранительная деятельность и обеспечение безопасности государства'
cofog_.loc[40,'name2'] = 'Промышленность, энергетика и строительство'
cofog_.loc[24,'name2'] = 'Сельское хозяйство и рыболовство'
cofog_.loc[32,'name2'] = 'Охрана окружающей природной среды и природных ресурсов, гидрометеорология, картография и геодезия'
cofog_.loc[22,'name2'] = 'Транспорт, связь и информатика'
cofog_.loc[31,'name2'] = 'Развитие рыночной инфраструктуры'
cofog_.loc[1,'name2'] = 'Государственное управление и местное самоуправление'
cofog_.loc[53,'name2'] = 'Культура, искусство и кинематография'
cofog_.loc[56,'name2'] = 'Средства массовой информации'
cofog_.loc[50,'name2'] = 'Здравоохранение и физическая культура'
cofog_.loc[10,'name2'] = 'Военная реформа'
cofog_.loc[7,'name2'] = 'Обслуживание государственного и муниципального долга'
cofog_.loc[27,'name2'] = 'Дорожное хозяйство'
cofog_.loc[78,'name2'] = 'Пополнение государственных запасов и резервов'
cofog_.loc[8,'name2'] = 'Финансовая помощь бюджетам других уровней'
cofog_.loc[31,'name2'] = 'Исследование и использование космического пространства'
cofog_.loc[23,'name2'] = 'Развитие рыночной инфраструктуры'

#cofog_.loc[32,'name2'] = 



In [99]:
cofog_[['code','name','name2']].to_excel('cofog_rus_mod1.xlsx', index=False)